In [1]:
from keras.layers.core import Activation, Dense, Dropout, SpatialDropout1D
from keras.layers import concatenate,Reshape,Add,LSTM,Multiply
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Bidirectional, LeakyReLU
from keras.models import Sequential,Model
from keras.optimizers import Adam

from keras import Model
from keras import Input

import cython
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from time import time
from gensim.models import Word2Vec
from keras.initializers import Constant
from keras.utils import to_categorical
from keras.preprocessing.text import text_to_word_sequence,Tokenizer
from keras.preprocessing.sequence import pad_sequences,TimeseriesGenerator

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 


Using TensorFlow backend.


# Внимание! 

Если планируете запускать весь ноутбук, то убедитесь, что количество озу в вашем пека >=32 гб. Либо коллаб в помощь

### Загрузим данные

размер словаря - 84267 слов,количество предложений - 73477, максимальная длинна текста - 398 символов, максимальное количество слов в предложении - 190


In [2]:
data=np.array(pd.read_csv('data.csv'))

### Проведем токенизацию

То есть разобьем исходные предложения на слова

In [3]:
def processText(data):
    tokens=[]
    for line in data:
        newToken=text_to_word_sequence(text=line[2],filters='!"#$%&amp;()*+,-./:;&lt;=>?@[\\]^_`{|}~\t\n\ufeff',
                                  lower=True,split=' ')
        tokens.append(newToken)
    return tokens

In [4]:
wordLists=processText(data)
print(wordLists[0])

['любые', 'разногласия', 'во', 'мнениях', 'скоро', 'улягутся', 'а', 'вы', 'продолжайте', 'делать', 'как', 'делали', 'но', 'постарайтесь', 'не', 'наступать', 'на', 'ноги', 'слишком', 'многим', 'иначе', 'ваши', 'сегодняшние', 'действия', 'сыграют', 'против', 'вас', 'в', 'будущем']


### Заменим слова в предложениях на соответсвующие в словаре индексы

In [5]:
num_words = 84267
tokenizer = Tokenizer(
    num_words=num_words,
    filters='!"#$%&amp;()*+,-—./:;&lt;=>?@[\\]^_`{|}~\t\n\xa0\ufeff',
    lower=True,
    split=' ',
    char_level=False) 

tokenizer.fit_on_texts(wordLists) 
sequences = np.array(tokenizer.texts_to_sequences(wordLists))
print(sequences[0])

[293, 448, 66, 8385, 4715, 15933, 22, 8, 10529, 336, 34, 10057, 25, 127, 3, 16752, 7, 12640, 169, 755, 254, 43, 3549, 279, 4619, 953, 12, 2, 614]


### Зафиксируем характеристики выборки

In [6]:
sentLen = len(max(sequences, key = len)) # max_len
word2index = tokenizer.word_index       #word_index
wordsNum = len(tokenizer.word_index) + 1 # num_words
embeddingDim=300

### Дополним предожения нулями до одной длины

In [7]:
sequences = pad_sequences(sequences = sequences, maxlen = sentLen,padding='post')
print(sequences[0])
print(sequences[0].shape)
print(sequences.shape)

[  293   448    66  8385  4715 15933    22     8 10529   336    34 10057
    25   127     3 16752     7 12640   169   755   254    43  3549   279
  4619   953    12     2   614     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [8]:
w2v = Word2Vec.load("word2vec.model")
embeddingMatrix = np.zeros((wordsNum, embeddingDim))
for word, i in tokenizer.word_index.items():
    if i > wordsNum: #если индекс превышает кол-во слов в словаре, то скипаем  
        continue
    embeddingVector = w2v[word] #получаем вектор соответствущий слову в модели word2vec
    if embeddingVector is not None:  #если слово отсутствует в словаре word2vec, то оно в матрице np.zeroes останется равным 0
        embeddingMatrix[i] = embeddingVector #если слово найдено в словаре токенизатора, то в embedding_matrix проставляем вектор соответствующий слову

C:\Users\Администратор\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


### Отнормируем вектора embedding'ов. 

In [9]:
scaler=MinMaxScaler((-1,1))

До нормализации

In [10]:
embeddingMatrix[458][:40]

array([ 2.20851088,  6.9667182 ,  1.03406489, -0.06461832,  1.46351445,
       -1.40225852, -0.97575563,  1.09570146,  1.00645208,  1.2965281 ,
        1.28454924, -2.3900702 ,  0.71035409,  0.67477763,  0.96809483,
        0.66907966, -0.6290099 ,  1.80262721, -0.87476927, -0.26172397,
        0.66931254,  4.15037966,  1.68157887,  0.67298663, -2.78617191,
        1.35166252,  0.58126897, -0.9560746 , -1.56313276,  0.81927693,
       -0.66007751, -2.74609256, -1.79234636,  1.8601526 ,  5.47511339,
        1.59147608, -1.3512764 ,  1.73950148,  2.29833245,  0.67214918])

In [11]:
embeddingMatrix=scaler.fit_transform(embeddingMatrix)

После нормализации

In [12]:
embeddingMatrix[458][:40]

array([ 0.15737807,  0.44272599, -0.03178928,  0.08572773,  0.03299629,
       -0.10667435,  0.05800479,  0.07516103,  0.10402152,  0.1108683 ,
        0.04387122, -0.11529571,  0.04801753,  0.08852304,  0.1785953 ,
        0.05145529, -0.12420372,  0.07294531, -0.14831744, -0.02330168,
       -0.01659107,  0.17392238,  0.12742558,  0.06224952, -0.14797107,
        0.05685835,  0.06767316, -0.0649765 , -0.11510154,  0.01222805,
       -0.10834389, -0.15692699, -0.17532073,  0.15444414,  0.26478314,
        0.16785802,  0.00540678,  0.12133913,  0.17657236,  0.09729453])

### Приведем датасет к специальному виду

 Например для текста "The sky was falling due to apocalypse " будет
 
 "The sky" -> was
 
 "sky was" -> falling
 
 " was falling " -> due 
 
 " falling due" -> to
 
 "due to " -> apocalypse
 


In [14]:
def word2idx(word):
    return w2v.wv.vocab[word].index
def idx2word(idx):
    return w2v.wv.index2word[idx]

In [15]:
seqlen=10
step=1
inputSent = []
labelSent = []
for i,line in enumerate(sequences):
    for j in range(0, len(line) - 1, step):
        inputSent.append(line[j])
        labelSent.append(line[j + 1])

In [16]:
inputSent[0:5]

[293, 448, 66, 8385, 4715]

In [17]:
labelSent[0:4]


[448, 66, 8385, 4715]

Соотнесем кадому слову из выборки Y вектор embedding'а

In [18]:
X=np.array(inputSent)
Y=np.zeros((X.shape[0],embeddingDim),dtype='float32')

for i,index in enumerate(labelSent):
    Y[i]=w2v[idx2word(index)]

C:\Users\Администратор\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [19]:
Y=scaler.fit_transform(Y)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2,shuffle=False) #, random_state = 42)

In [23]:
del X

In [24]:
del Y

In [ ]:
del x_train
del y_train
del x_test
del y_test

In [ ]:
np.save('x_train.npy',x_train)
np.save('y_train.npy',y_train)
np.save('x_test.npy',x_test)
np.save('y_test.npy',y_test)

## Раздел "спасите, не работает"

Поставлена задача по входному эмбеддингу слова определить эмбеддинг следующего слова, то есть - задача регресси. Однако по факту ничего не работает. 

Либо модель плохая (да), либо оптимизатор плохой (да) и все в таком духе.

Варианты с GRU, SLTM не работают.

Протестированы регрессионные функции потерь: косинусная метрика, среднеквадратичная 

In [25]:
modelGRU = Sequential()
#embedding_layer = w2v.wv.get_keras_embedding(train_embeddings=False)
modelGRU.add(Embedding(input_dim = num_words+1, 
                              output_dim = embeddingDim, 
                              embeddings_initializer = Constant(embeddingMatrix),
                              input_length =1, 
                              trainable = False))
#modelGRU.add(Embedding(num_words, embedding_size))
#modelGRU.add(SpatialDropout1D(0.2))
#modelGRU.add(Bidirectional(GRU(40, return_sequences=True)))
#modelGRU.add(Bidirectional(LSTM(128)))
#modelGRU.add(LSTM(8,return_sequences=True ))
modelGRU.add(LSTM(128))
#modelGRU.add(Dropout(0.2))

modelGRU.add(Dense(64))
modelGRU.add(LeakyReLU(0.2))

modelGRU.add(Dense(embeddingDim,activation = 'tanh'))
modelGRU.summary()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 300)            25280400  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               19500     
Total params: 25,527,804
Trainable params: 247,404
Non-trainable params: 25,280,400
_________________________________________________________________


In [ ]:
modelGRU.compile(loss="cosine_similarity", optimizer='adam')
historyGRU = modelGRU.fit(x_train, y_train, batch_size=16384, epochs=25, validation_split=0.2, verbose=1)

Train on 8887777 samples, validate on 2221945 samples
Epoch 1/25
8887777/8887777 [==============================] - 77s 9us/step - loss: -0.8064 - val_loss: -0.8097
Epoch 2/25
8887777/8887777 [==============================] - 73s 8us/step - loss: -0.8088 - val_loss: -0.8105
Epoch 3/25
8887777/8887777 [==============================] - 73s 8us/step - loss: -0.8099 - val_loss: -0.8111
Epoch 4/25
8887777/8887777 [==============================] - 72s 8us/step - loss: -0.8106 - val_loss: -0.8113
Epoch 5/25
2703360/8887777 [========>.....................] - ETA: 45s - loss: -0.8110

In [ ]:
x_train.shape

In [ ]:
y_train[0]